## Importing Packages 

In [1]:
#we import these packages and link to our tesseract executable file
import cv2
import pytesseract
import numpy as np
from PIL import ImageGrab
import time
import os

# for data appending
import re
import time

### Setting Parameters

In [2]:
per = 1

### Query Image Region Co-ordinates

In [3]:
roi= [[(266, 96), (602, 138), 'text', 'DL No'], 
      [(788, 94), (990, 146), 'text', 'Date of Issue'], 
      [(308, 128), (498, 178), 'text', 'Valid Till'], 
      [(372, 288), (514, 416), 'text', 'COV'], 
      [(494, 452), (704, 508), 'text', 'DOB'], 
      [(804, 452), (878, 502), 'text', 'Blood Group'], 
      [(158, 498), (718, 546), 'text', 'Name'], 
      [(174, 542), (720, 572), 'text', 'S/D/W'], 
      [(100, 572), (1000, 610), 'text', 'Address 1'], 
      [(24, 602), (1000, 640), 'text', 'Address 2'], 
      [(30, 636), (900, 678), 'text', 'City'], 
      [(90, 670), (226, 710), 'text', 'Pin'], 
      [(302, 732), (410, 784), 'text', 'Issuing Authority']]

### Importing PyTesseract

In [4]:
pytesseract.pytesseract.tesseract_cmd = 'C://Program Files//Tesseract-OCR//tesseract.exe'

### Importing Database

In [5]:
img = cv2.imread("C://Users//Arvind Desai//Desktop//Sem 6 IDS Project//Query//Dharmi Final Query.jpg") 
h,w,c = img.shape
#img = cv2.resize(img,(w//3,h//3))
#cv2.imshow("Output", img)
#cv2.waitKey(0)

orb = cv2.ORB_create(5000)
kp1, des1 = orb.detectAndCompute(img,None)
impKp1 = cv2.drawKeypoints(img,kp1,None)
# cv2.imshow("KeyPointsQuery", impKp1)
# cv2.waitKey(0)

path = "C://Users//Arvind Desai//Desktop//Sem 6 IDS Project//Final License Database"
myPicList = os.listdir(path)
print(myPicList)

['Aaditya.jpeg', "Aman's Dad.jpg", "Aman's Sister.jpg", 'Amit.jpeg', 'Atul.jpeg', 'Bipin.jpeg', 'BK.jpeg', 'Charmi.jpeg', 'Chetan.jpeg', 'Chitranjan.jpeg', 'Darshan.jpeg', 'Dhairya.jpeg', 'Dharmi photo.jpg', 'Dhaval.jpeg', 'Dhruv.jpg', 'Dipika.jpeg', 'Divya.jpeg', 'Garv.jpeg', 'Hardik.jpeg', 'Hasti.jpeg', 'Heta.jpeg', 'Khushi.jpg', "Mahi's Father.jpeg", 'Manav.jpg', 'Mehali.jpeg', 'Mek.jpeg', 'Milind.jpeg', 'Milonee.jpeg', 'Neel.jpg', 'Nia.jpg', 'Nixit.jpeg', 'Palash.jpg', 'Param.jpeg', 'Piyushkumar.jpeg', 'R Shriram.jpeg', 'Rajesh Dubey.jpeg', 'Reena.jpeg', 'Rutvi.jpeg', 'Saisha.jpg', 'Shanay.jpeg', 'Shashank Ranka Demo.jpeg', 'Shlok.jpeg', 'Smit.jpeg', 'Tejas.jpg', 'Vanshika.jpeg', 'Vicky.jpeg', 'Vinayak.jpeg']


### Defining an Image Pre-Processing Function

In [6]:
def increase_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

# Text Recognition & Database Creation

In [7]:
start = time.time()

for j,y in enumerate(myPicList):
    img1 = cv2.imread(path +"//"+y)
    #img1 = cv2.resize(img1, (w // 3, h // 3))
    #cv2.imshow(y, img1)
    
    kp2, des2 = orb.detectAndCompute(img1,None)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING)
    matches = bf.match(des2,des1)
    matches.sort(key= lambda x: x.distance)
    good = matches[:int(len(matches)*(per/100))]
    imgMatch = cv2.drawMatches(img1,kp2,img,kp1,good[:100],None,flags=2)
    #imgMatch = cv2.resize(imgMatch, (w // 3, h // 3))
    #cv2.imshow(y, imgMatch)
    #cv2.waitKey(0)
    
    srcPoints = np.float32([kp2[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dstPoints = np.float32([kp1[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

    M, _ = cv2.findHomography(srcPoints,dstPoints,cv2.RANSAC,5.0)
    imgScan = cv2.warpPerspective(img1,M,(w,h))
    #imgScan = cv2.resize(imgScan, (w // 3, h // 3))
    #cv2.imshow(y, imgScan)
    #cv2.waitKey(0)
    
    imgScan = increase_brightness(imgScan, value=63)
    
    kernel = np.ones((2,2),np.uint8)
    imgScan = cv2.dilate(imgScan,kernel,iterations = 1)
#     imgScan = cv2.erode(imgScan,kernel,iterations = 1)
   
    imgShow = imgScan.copy()
    imgMask = np.zeros_like(imgShow)

    myData = []

    print(f'################## Extracting Data from License {j+1}  ##################')

    for x,r in enumerate(roi):
        cv2.rectangle(imgMask, (r[0][0],r[0][1]),(r[1][0],r[1][1]),(0,255,0),cv2.FILLED)
        imgShow = cv2.addWeighted(imgShow,0.99,imgMask,0.1,0)
#     imgShow = cv2.resize(imgShow, (w // 3, h // 3))
#     cv2.imshow(y+"2", imgShow)
        imgCrop = imgScan[r[0][1]:r[1][1], r[0][0]:r[1][0]]
        ##cv2.imshow(str(x), imgCrop)
#         print(f'{r[3]} :{pytesseract.image_to_string(imgCrop)}')
        myData.append(pytesseract.image_to_string(imgCrop, lang='eng',config='--psm 12 --oem 3'))
##       cv2.putText(imgShow,str(myData[x]),(r[0][0],r[0][1]), cv2.FONT_HERSHEY_PLAIN,2.5,(0,0,255),4)
    
    with open('LicenseOutput.csv','a+') as f:
        for data in myData:
            f.write(re.sub(r"[\n\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff\.\,\'\?\=\!\|\>\_\$\~\[\:\]\'\`\*\‘\—\)\(\’\"\™\“\€]", '', data)+',')
        f.write('\n') 
      
 
#     print(myData)
time.sleep(1)
# end time
end = time.time()

# total time taken
print(f"\nRuntime of the program is {int((end - start)/60)} min {(end - start)-int(end - start)} sec")
#     cv2.imshow(y+"2", imgShow)
#     cv2.imwrite(y,imgShow)
# cv2.waitKey(0)

################## Extracting Data from License 1  ##################
################## Extracting Data from License 2  ##################
################## Extracting Data from License 3  ##################
################## Extracting Data from License 4  ##################
################## Extracting Data from License 5  ##################
################## Extracting Data from License 6  ##################
################## Extracting Data from License 7  ##################
################## Extracting Data from License 8  ##################
################## Extracting Data from License 9  ##################
################## Extracting Data from License 10  ##################
################## Extracting Data from License 11  ##################
################## Extracting Data from License 12  ##################
################## Extracting Data from License 13  ##################
################## Extracting Data from License 14  ##################
###############